In [1]:
# import libraries
from docxtpl import DocxTemplate
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_SECTION, WD_ORIENTATION
from docx.shared import RGBColor, Mm, Pt, Inches
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.table import WD_ROW_HEIGHT_RULE
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib.parse import urljoin
import json
from datetime import date, datetime as dt

c:\Users\Gabriel O\Desktop\ACL\ACL\Knowledge Base\Python_Reports\report_temp\Lib\site-packages\docxcompose\properties.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
####################################################################################
# fetch data
####################################################################################

BEARER_TOKEN = 'd358c3e1709f951fe611415e7a0b1956b783127d52d541adc48f649761408f15'
org_id = '11534' #'52734'
id = '492704'
server_url = 'https://apis-us.diligentoneplatform.com'
BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects'
headers = {'Authorization' : f'Bearer {BEARER_TOKEN}', 'Content-Type': 'application/vnd.api+json'}
params = {'filter[status]':'active', 'filter[start_date][lte]': dt.today().isoformat(), 'include':'project, project.fieldwork'}

def fetch_data(url, params=None, max_retries=5, timeout=10, backoff_factor=1):
    """
        fetch data from endpoint with maxixum retries, timeout and session management

        Args:
            url(str): targeted endpoint
            max_retries(int): no of retry attempt upon non-response from the server
            timeout(int): prevent prolonged server connection
            backoff_factor = wait multiplier between request
    """
    retries = Retry(total= max_retries, 
                    backoff_factor=backoff_factor, 
                    status_forcelist=[429, 500, 502, 503, 504], #retries on these http errors
                    allowed_methods=['GET'])
    session = requests.session()
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://',adapter)
    session.mount('https://', adapter)
    all_data = []
    while url:
        try:
            resp = session.get(url, headers=headers, params=params, timeout=10)
            resp.raise_for_status()
        except Exception as e:
            print(f'failure to fetch data: {e}')
        project_data = resp.json().get('data', [])
        all_data = all_data + project_data if isinstance(project_data, list) else project_data # check and accumulate multiple project or return a single targerted project
        next_page = resp.json()['links']['next'] if isinstance(project_data, list) else None
        url = urljoin(server_url,next_page) if next_page else None
        params = None
    session.close()
    return all_data

In [3]:
# fetch valid project data ***************************************************************
projects = fetch_data(BASE_URL)

valid_projects = []
for project in projects:
    sd = project.get("attributes", {}).get("start_date", "")
    try:
        if dt.strptime(sd, "%Y-%m-%d").date() <= date.today():
            valid_projects.append(project)
    except ValueError:
            continue

valid_projects, len(valid_projects)



([{'id': '17702',
   'type': 'projects',
   'attributes': {'name': 'Risk_Assessment',
    'state': 'active',
    'status': 'active',
    'created_at': '2015-02-17T10:54:17Z',
    'updated_at': '2022-05-19T12:37:45Z',
    'description': 'sdsg',
    'background': None,
    'budget': 200,
    'position': 0,
    'certification': False,
    'control_performance': True,
    'risk_assurance': False,
    'management_response': None,
    'max_sample_size': 0,
    'number_of_testing_rounds': 0,
    'opinion': None,
    'opinion_description': None,
    'purpose': None,
    'scope': None,
    'start_date': '2022-05-01',
    'target_date': '2022-06-08',
    'custom_attributes': [],
    'progress': 16.7,
    'time_spent': 2.0,
    'planned_start_date': None,
    'planned_end_date': None,
    'planned_milestone_date': None,
    'actual_start_date': None,
    'actual_end_date': None,
    'actual_milestone_date': None,
    'tag_list': []},
   'relationships': {'project_type': {'data': {'id': '64148',
 

In [ ]:
# fetch planning files data for specied project ***************************************************************
project_plannings = {}
working_project = [project for project in valid_projects if project['id']==id] # filter for project of interest.
for project in working_project:
    project_id = project['id']
    BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{project_id}/planning_files'
    plannings = fetch_data(BASE_URL, params=None)
    project_plannings[project_id] = plannings
project_plannings, len(project_plannings)

({'492704': []}, 1)

In [5]:
# fetch result files data **************************************************************
project_results = {}
for project in working_project:
    project_id = project['id']
    BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{project_id}/results_files'
    results = fetch_data(BASE_URL, params=None)
    project_results[project_id] = results
project_results, len(project_results)

({'492704': []}, 1)

In [6]:
# fetch objective data **************************************************************
project_objectives = {}
for project in working_project:
    project_id = project['id']
    BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/projects/{project_id}/objectives'
    objectives = fetch_data(BASE_URL, params=None)
    project_objectives[project_id] = objectives
project_objectives, len(project_objectives)

({'492704': [{'id': '1896423',
    'type': 'objectives',
    'attributes': {'title': 'Operations Department',
     'description': '',
     'reference': '01',
     'division_department': '',
     'owner': '',
     'executive_owner': '',
     'position': 1,
     'custom_attributes': [],
     'created_at': '2025-08-26T12:57:08Z',
     'updated_at': '2025-08-26T12:57:08Z',
     'risk_control_matrix_id': '1895050',
     'walkthrough_summary_id': '1896238',
     'test_plan_id': '1884576',
     'testing_round_1_id': '1260330',
     'testing_round_2_id': None,
     'testing_round_3_id': None,
     'testing_round_4_id': None},
    'relationships': {'project': {'data': {'id': '492704',
       'type': 'projects'}},
     'assigned_user': {'data': {'id': 'SZeo97qjNcBnDniLQnWd',
       'type': 'users'}},
     'owner_user': {'data': None},
     'executive_owner_user': {'data': None}}}]},
 1)

In [7]:
# fetch risk data **************************************************************
## data structure: {project_id: objective_id: [risk items]}
project_objectives_risks = {}
for project in working_project:
    project_id = project['id']
    project_objectives_risks[project_id] = {}   # dict to hold object-risk data for each project
    for objective in project_objectives[project_id]:
        objective_id = objective['id']
        if objective_id:
            BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/objectives/{objective_id}/risks'
            risks = fetch_data(BASE_URL, params=None)
            project_objectives_risks[project_id][objective_id] = risks
project_objectives_risks, len(project_objectives_risks)

({'492704': {'1896423': [{'id': '8232794',
     'type': 'risks',
     'attributes': {'title': 'Non-adherence to approved Preventive Maintenance Plan',
      'description': '<p>Non-adherence to approved Preventive Maintenance Plan (PMP) of critical assets that might compromise the safety of the operations\xa0\xa0</p>',
      'risk_id': 'R1',
      'position': 1,
      'impact': 'Major',
      'likelihood': 'Likely',
      'owner': '',
      'custom_attributes': [{'id': '151076',
        'term': 'Site',
        'value': [],
        'required': False,
        'default_values': []}],
      'custom_factors': [],
      'created_at': '2025-08-26T13:18:09Z',
      'updated_at': '2025-08-26T13:18:09Z'},
     'relationships': {'objective': {'data': {'id': '1896423',
        'type': 'objectives'}}}},
    {'id': '8232795',
     'type': 'risks',
     'attributes': {'title': 'Incompleteness of the PMP may result in rejected claims',
      'description': '<p>Incompleteness of the PMP may result in re

In [8]:
# fetch control data **************************************************************
## data structure: {project_id: objective_id: [controls]}
project_objectives_controls = {}
for project in working_project:
    project_id =  project['id'] 
    project_objectives_controls[project_id] = {}   # dict to hold object-risk data for each project
    for objective in project_objectives[project_id]:
        objective_id = objective['id']
        if objective_id:
            BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/objectives/{objective_id}/controls'
            controls = fetch_data(BASE_URL, params=None)
            project_objectives_controls[project_id][objective_id] = controls
project_objectives_controls, len(project_objectives_controls)

({'492704': {'1896423': [{'id': '14421648',
     'type': 'controls',
     'attributes': {'title': 'Training and experience of support',
      'description': '<p>Training and experience of support is pivotal for the asstst that don’t have manuals or reference documents to an extent that when referenced, there was not significant deviation in some cases\xa0</p>',
      'control_id': '01',
      'owner': '',
      'position': 1,
      'created_at': '2025-08-26T13:18:09Z',
      'updated_at': '2025-08-26T13:18:09Z',
      'custom_attributes': [],
      'audit_is_workplan': False},
     'relationships': {'objective': {'data': {'id': '1896423',
        'type': 'objectives'}}},
     'links': {'ui': 'https://cts.projects.diligentoneplatform.com/audits/492704/objectives/1896423/controls/14421648'}},
    {'id': '14421649',
     'type': 'controls',
     'attributes': {'title': 'Random inconsistent scheduled visits',
      'description': '<p>Random inconsistent scheduled visits to these sites for 

In [9]:
# fetch control test data **************************************************************
## data structure: {project_id: objective_id: control_id: [control_test]}
### a control can have one or more testing data depending on the testing rounds
project_objectives_controls_tests = {}
for project in project_objectives_controls:
    project_id = project
    project_objectives_controls_tests[project_id] = {}   # dict to hold object-risk data for each project
    for objective in project_objectives_controls[project_id]:
        objective_id = objective
        project_objectives_controls_tests[project_id][objective_id]={}
        for control in project_objectives_controls[project_id][objective_id]:
            control_id = control['id']
            params = {'filter[project.id]':project_id, 'filter[control.id]':control_id} # filter for project and specified control
            BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/control_tests'
            control_tests = fetch_data(BASE_URL, params=params)
            project_objectives_controls_tests[project_id][objective_id][control_id] = control_tests
project_objectives_controls_tests, len(project_objectives_controls_tests)


({'492704': {'1896423': {'14421648': [{'id': '9882762',
      'type': 'control_tests',
      'attributes': {'assignee_name': None,
       'testing_round_number': 1,
       'not_applicable': False,
       'sample_size': None,
       'testing_results': None,
       'testing_conclusion': False,
       'testing_conclusion_status': 'Partially Operating Effectively',
       'custom_attributes': [],
       'created_at': '2025-08-26T13:18:09Z',
       'updated_at': '2025-08-26T13:57:34Z'},
      'relationships': {}}],
    '14421649': [{'id': '9882763',
      'type': 'control_tests',
      'attributes': {'assignee_name': None,
       'testing_round_number': 1,
       'not_applicable': False,
       'sample_size': None,
       'testing_results': None,
       'testing_conclusion': False,
       'testing_conclusion_status': 'Operating Ineffectively',
       'custom_attributes': [],
       'created_at': '2025-08-26T13:18:10Z',
       'updated_at': '2025-08-26T14:29:04Z'},
      'relationships': {}}

In [10]:
# fetch control design data from walkthroughs *************************************
## data structure: {project_id: objective_id: control_id: [walkthroughs]}
### a control can have one or more testing data depending on the testing rounds
project_objectives_controls_walkthroughs = {}
for project in project_objectives_controls:
    project_id = project
    project_objectives_controls_walkthroughs[project_id] = {}   # dict to hold object-risk data for each project
    for objective in project_objectives_controls[project_id]:
        objective_id = objective
        project_objectives_controls_walkthroughs[project_id][objective_id]={}
        for control in project_objectives_controls[project_id][objective_id]:
            control_id = control['id']
            params = {'filter[project.id]':project_id, 'filter[control.id]':control_id} # filter for project and specified control
            BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/walkthroughs'
            walkthrough = fetch_data(BASE_URL, params=params)
            project_objectives_controls_walkthroughs[project_id][objective_id][control_id] = walkthrough
project_objectives_controls_walkthroughs, len(project_objectives_controls_walkthroughs)

({'492704': {'1896423': {'14421648': [{'id': '10357901',
      'type': 'walkthroughs',
      'attributes': {'walkthrough_results': '<p>Inadequate design \xa0</p>',
       'control_design': False,
       'created_at': '2025-08-26T13:18:09Z',
       'updated_at': '2025-08-26T13:25:33Z',
       'custom_attributes': []},
      'relationships': {}}],
    '14421649': [{'id': '10357902',
      'type': 'walkthroughs',
      'attributes': {'walkthrough_results': None,
       'control_design': None,
       'created_at': '2025-08-26T13:18:10Z',
       'updated_at': '2025-08-26T13:18:10Z',
       'custom_attributes': []},
      'relationships': {}}],
    '14421650': [{'id': '10357903',
      'type': 'walkthroughs',
      'attributes': {'walkthrough_results': None,
       'control_design': None,
       'created_at': '2025-08-26T13:18:10Z',
       'updated_at': '2025-08-26T13:18:10Z',
       'custom_attributes': []},
      'relationships': {}}],
    '14421651': [{'id': '10357904',
      'type': 'wal

In [4]:
# fetch issues/findings data *************************************************
## targeted filters for projects, objectives, control, control test, risk

params = {'filter[project.id]':id}

BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/issues'
issues = fetch_data(BASE_URL, params=params)

issues, len(issues)

([{'id': '1101239',
   'type': 'issues',
   'attributes': {'title': 'Lack of user manual or reference documents for some critical equipment ',
    'description': '<p>Lack of user manuals or reference documents for some critical assets which provide guidance into their preventive maintenance. Examples of such equipment include:</p><p>Windward system for Impact repot mapping</p><p>This was mainly due to oversight.\xa0</p>',
    'creator_name': 'Dinah Maina',
    'created_at': '2025-08-26T13:41:18Z',
    'updated_at': '2025-08-28T12:04:00Z',
    'position': 1,
    'owner': 'Manager Support',
    'recommendation': '<p>Technicians/ Support should keep user manuals and or reference documents in the absence of the manual (such as international standards) for equipment after installation for reference when developing and implementing PMP.\xa0</p>',
    'deficiency_type': 'Finding',
    'severity': None,
    'published': True,
    'identified_at': '2025-08-26T13:36:12Z',
    'reference': '1',
 

In [14]:
BASE_URL = f'https://apis-us.diligentoneplatform.com/v1/orgs/{org_id}/frameworks'
frameworks = fetch_data(BASE_URL)
frameworks , len(frameworks)

([{'id': '245399',
   'type': 'frameworks',
   'attributes': {'name': 'Anti Bribery and Corruption',
    'created_at': '2023-08-18T12:28:50Z',
    'updated_at': '2024-06-03T08:06:20Z',
    'description': '',
    'background': '<p>Anti-bribery and anti-corruption risks are not well mitigated by standard (i.e. reporting focused) financial controls. The impact of non-compliance to anti-bribery and anti-corruption legislation is potential investigation by authorities leading to fines up to billions.</p>\r\n\r\n<p>Organizations with operations in the United States are subject to the Foreign Corrupt Practices Act, and organizations connected with the United Kingdom are subject to the U.K. Bribery Act.</p>\r\n\r\n<p>The <strong>Foreign Corrupt Practices Act</strong> (FCPA) of 1977 is a US Federal Law primarily intended to prohibit payments of bribes to foreign officials and political figures (also know as the Anti-Bribery Provision). The second provision requires that companies maintain trans